# Prompt Engineering with Large Language Models (e.g. GPT)

The goal of this exercise is to help you understand the concept of prompt engineering with large language models, such as GPT-3, and practice creating effective prompts for specific tasks or information retrieval.

This exercise also incorporates retrieval augmented generation (RAG) techniques. You will practice creating effective prompts that involve retrieving information and generating coherent responses.

For a deeper dive, checkout the [Hugging Face NLP Course](https://huggingface.co/learn/nlp-course/chapter0/1).


## Setup

**IMPORTANT: You need to run the Code in each section, in the order that it appears, each time you connect to a runtime.**

To get started with this notebook, click "File", and "Save a copy" to a location of your choosing.

### Install dependencies

In [ ]:
# install langchain, which we'll use to run
# interactions with LLMs
!pip install --upgrade langchain

# install the LLMs we're going to use
!pip install gpt4all openai

# install bs4 (BeautifulSoup) for web loading
!pip install bs4

# install pypdf for PDF reading
!pip install pypdf

# install the document embedding and indexing tools
!pip install faiss-cpu huggingface-hub sentence_transformers

# install other dependencies
!pip install cohere tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.3/220.3 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filena

### Utilities

In [ ]:
import time

"""
UTILS
"""

def now_ms(get_time=time.time):
    """
    Returns the current time in milliseconds.
    """
    return round(get_time() * 1000)


def duration_ms(start_time, now_ms=now_ms):
    """
    Returns the duration in milliseconds since the given start time.
    """
    return now_ms() - start_time


def duration_s(start_time, now_ms=now_ms):
    """
    Returns the duration in seconds since the given start time.
    """
    return duration_ms(start_time, now_ms) / 1000


SHOULD_TRACE = True
START_TIME = now_ms()


def trace(*args, **kwargs):
    """
    Prints a trace message if the `SHOULD_TRACE` flag is set to True.
    """
    if SHOULD_TRACE:
        print(f"TRACE::{duration_s(START_TIME)}::", *args, **kwargs)


### Index Loading

In [ ]:
import os

from langchain.embeddings import (GPT4AllEmbeddings, HuggingFaceEmbeddings,
                                  OpenAIEmbeddings)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

def use_huggingface_embeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
):
    """
    Prepares the HuggingFace embeddings.
    """
    return HuggingFaceEmbeddings(model_name=model_name)


def use_openai_embeddings():
    """
    Prepares the OpenAI embeddings.
    """
    return OpenAIEmbeddings()


def use_gpt4all_embeddings():
    """
    Prepares the GPT4All embeddings.
    """
    return GPT4AllEmbeddings()


def split_and_save_index(
    documents,
    store_path,
    make_embeddings=use_gpt4all_embeddings,
):
    """
    Parses the loader output and stores it in a local vector store.

    PARAMETERS:
    documents (list):  The documents to parse.
    store_path (str):  The path to the local vector store to
                       save the parsed PDF to.
    model_name (str):  The name of the HuggingFace model to use
                       for generating embeddings.

    RETURNS:
    (FAISS): The loaded index.
    """
    trace("splitting source")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024, chunk_overlap=64)
    texts = text_splitter.split_documents(documents)
    trace("preparing embeddings")
    embeddings = make_embeddings()
    trace("indexing source")
    faiss_index = FAISS.from_documents(texts, embeddings)
    faiss_index.save_local(store_path)
    return faiss_index


def load_index_from_cache(
    store_path,
    model_name="sentence-transformers/all-MiniLM-L6-v2",
):
    """
    Loads the embeddings and index from a local vector store.

    PARAMETERS:
    store_path (str):  The path to the local vector store to
                       load the embeddings and index from.

    RETURNS:
    (FAISS): The loaded index.
    """
    trace("preparing embeddings")
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    trace("loading index")
    faiss_index = FAISS.load_local(store_path, embeddings)
    return faiss_index


def load_index(index_path, make_loader):
    """
    Loads the index from the given path. If the index doesn't exist,
    it will be created and cached.

    PARAMETERS:
    index_path (str): The path to the index.
    make_loader (function): A function that returns a document loader.

    RETURNS:
    (FAISS): The loaded index.
    """
    path_exists = os.path.isdir(index_path)

    if path_exists is False:
        trace(f"loading new index {index_path}")
        split_and_save_index(
            make_loader().load_and_split(),
            index_path,)

    trace(f"using cached index {index_path}")
    return load_index_from_cache(index_path)


### LLMs

#### GPT4All

[GPT4All](https://gpt4all.io/index.html) is a free-to-use, locally running, privacy-aware chatbot. No GPU or internet required.

You can use GPT4All in the same way that you would chatGPT or OpenAI GPT APIs.

This function produces a [chainable GPT4All LLM](https://python.langchain.com/docs/integrations/llms/gpt4all) that can be composed with [langchain LCEL chains](https://python.langchain.com/docs/expression_language).

In [ ]:
from gpt4all import GPT4All
from langchain import llms

def use_gpt4all(model_name, model_cache_path, fq_model_path):
    """
    Prepares a LLM for use in Q&A. Downloads the model to the
    cache if it isn't already there.

    PARAMETERS:
    model_name (str):       The name of the model to use.
    model_cache_path (str): The path to the model cache.
    fq_model_path (str):    The fully qualified path to the model.

    RETURNS:
    (Any (LCEL<LLM>)): The prepared LLM.
    """
    trace("preparing LLM")
    trace(f"model_name={model_name}")
    trace(f"model_cache_path={model_cache_path}")
    trace(f"fq_model_path={fq_model_path}")

    # if the chosen model isn't cached, this will
    # load it into the cache for future use
    # (~/.cache/gpt4all on linux & macos)
    GPT4All(model_name=model_name, model_path=model_cache_path)

    # load an LCEL chain wrapper of GPT4All
    return llms.GPT4All(model=fq_model_path)


#### OpenAI

[OpenAI](https://platform.openai.com/) is a pay-to-use, cloud running, exposed (not private) chatbot. No GPU is required, but you need an internet connection to use it.

You will need an API key to use OpenAI, which you can create on the [API Keys](https://platform.openai.com/account/api-keys) page after setting up your billing account.

This function produces a [chainable OpenAI LLM](https://python.langchain.com/docs/integrations/llms/gpt4all) that can be composed with [langchain LCEL chains](https://python.langchain.com/docs/expression_language).

In [ ]:
from getpass import getpass

from langchain import llms

def useOpenAI():
    """
    Prepares the OpenAI LLM. Prompts for the API key if it isn't
    already set in the environment.
    """
    OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

    if OPENAI_API_KEY is None:
        OPENAI_API_KEY = getpass(
            "Enter your OpenAI API key (from: https://platform.openai.com/account/api-keys)")

    return llms.OpenAI(openai_api_key=OPENAI_API_KEY)

## Choose your LLM

Two options for LLMs are provided: GPT4All and OpenAI. GPT4All is free, runs locally, and is private. However it is much slower than OpenAI. If you want faster responses and have free tokens, or are willing to spend $6+, you can choose to use OpenAI.

Once you choose which LLM you wish to use, make sure it is uncommented and that the other one is commented in the next code block, **and then run it**.

From this point on, in the document, use `llm` whenever you chain the LLM.

_NOTE If you choose GPT4All, this will take a while, It needs to download the model you choose for GPT4All. I chose a mini model by default to reduce the loading time_.

If you choose GPT4All, you can choose our model and there are a lot of choices. Here are some of the most effective options (from https://gpt4all.io/index.html, 2023-11):

<details>
  <summary>Best overall fast chat model: `mistral-7b-openorca.Q4_0.gguf` (click to expand)</summary>

- Fast responses
- Chat based model
- Trained by Mistral AI
- Finetuned on OpenOrca dataset curated via Nomic Atlas
- Licensed for commercial use
- SIZE: 3.83 GB
- RAM: 8 GB
</details>

<details>
  <summary>Best overall fast instruction following model: `mistral-7b-instruct-v0.1.Q4_0.gguf` (click to expand)</summary>

- Fast responses
- Trained by Mistral AI
- Uncensored
- Licensed for commercial use
- SIZE: 3.83 GB
- RAM: 8 GB
</details>

<details>
  <summary>Very fast model with good quality: `gpt4all-falcon-q4_0.gguf` (click to expand)</summary>

- Fastest responses
- Instruction based
- Trained by TII
- Finetuned by Nomic AI
- Licensed for commercial use
- SIZE: 3.92 GB
- RAM: 8 GB
</details>

<details>
  <summary>Best overall larger model: `wizardlm-13b-v1.2.Q4_0.gguf` (click to expand)</summary>

- Instruction based
- Gives very long responses
- Finetuned with only 1k of high-quality data
- Trained by Microsoft and Peking University
- Cannot be used commercially
- SIZE: 6.86 GB
- RAM: 16 GB
</details>

<details>
  <summary>Extremely good model: `nous-hermes-llama2-13b.Q4_0.gguf` (click to expand)</summary>

- Instruction based
- Gives long responses
- Curated with 300,000 uncensored instructions
- Trained by Nous Research
- Cannot be used commercially
- SIZE: 6.86 GB
- RAM: 16 GB
</details>

<details>
  <summary>Very good overall model: `gpt4all-13b-snoozy-q4_0.gguf` (click to expand)</summary>

- Instruction based
- Based on the same dataset as Groovy
- Slower than Groovy, with higher quality responses
- Trained by Nomic AI
- Cannot be used commercially
</details>


In [ ]:
def choose_gpt4all():
    GPT_MODEL_NAME = "orca-mini-3b-gguf2-q4_0.gguf"
    GPT_MODEL_CACHE_PATH = "models/" # os.path.expanduser("~/.cache/gpt4all/")
    GPT_FQ_MODEL_PATH = f"{GPT_MODEL_CACHE_PATH}{GPT_MODEL_NAME}"

    return use_gpt4all(GPT_MODEL_NAME, GPT_MODEL_CACHE_PATH, GPT_FQ_MODEL_PATH)

# if you choose to use gpt4all, you need to add a "models" directory
# in colab, click the folder icon on the left of the screen,
# right click in the filesystem palette, choose "New folder",
# and name it, "models"
llm = choose_gpt4all()
# llm = useOpenAI()

TRACE::245.111:: preparing LLM
TRACE::245.113:: model_name=orca-mini-3b-gguf2-q4_0.gguf
TRACE::245.113:: model_cache_path=models/
TRACE::245.113:: fq_model_path=models/orca-mini-3b-gguf2-q4_0.gguf


100%|██████████| 1.98G/1.98G [01:06<00:00, 29.7MiB/s]


## Reading 1: Zero-shot, Few-shot Prompting, and Retrieval Augmented Generation (RAG)

1. **Zero-shot prompting**:

   Zero-shot prompting is a technique used to interact with large language models, like GPT-3, without providing any specific training data or examples. In this approach, users can simply describe their task or question, and the model can generate a response based on its pre-existing knowledge and understanding of language. Zero-shot prompting doesn't require prior data or examples related to the specific query. Instead, it relies on the model's general understanding of language and its ability to generate contextually appropriate responses even for unseen topics or tasks. This makes it a versatile tool for various natural language understanding and generation tasks.

   **Example**: A user asks, "Explain the theory of relativity." In this zero-shot scenario, the model generates a coherent explanation of Einstein's theory of relativity without any pre-defined training data.

2. **Few-shot prompting**:

   Few-shot prompting is an extension of zero-shot prompting where users provide a small number of examples or context to guide the model's response. Instead of relying solely on the model's general knowledge, users give a few pieces of information or examples related to the task to help the model understand the context better. This approach is useful when the user wants to fine-tune the model's response for a specific task or topic, and it allows the model to adapt more precisely to the user's requirements.

   **Example**: A user asks, "Translate the following English text into Spanish: 'The quick brown fox jumps over the lazy dog.' Here are a few sample translations for reference: 'Hello, world: Hola Mundo,' 'The sun is shining: El sol está brillando,' 'It's a beautiful day: Es un hermoso día.'" The model uses these examples to generate a translation, even if it hasn't seen this specific input before.

3. **Retrieval Augmented Generation (RAG)**:

   Retrieval Augmented Generation (RAG) is a technique that combines the capabilities of large language models with information retrieval from external sources. It involves retrieving relevant information from a knowledge base or external documents and using that retrieved information to enhance the model's generated output. In RAG, the model first searches for and extracts information from a vast dataset or external sources, and then it generates a response that integrates this retrieved knowledge. This allows the model to provide more accurate, contextually rich, and factually grounded responses by leveraging external information.

   **Example**: When asked a question about a historical event, the model can search a database of historical documents to retrieve information and then generate a response that incorporates those historical facts.

In summary, zero-shot prompting relies on the model's pre-existing knowledge, few-shot prompting provides a limited amount of context or examples to fine-tune responses, and retrieval augmented generation involves an explicit retrieval step to gather information from external sources before generating a response. These techniques are valuable for a wide range of natural language processing tasks.

## Exercise 1: Create Effective Prompts

In this exercise, you will generate effective prompts for a large language model to perform specific tasks. There are several tasks for you to explore.



### Example 1: Translation

**Task**: Generate a prompt that instructs the language model to translate an English text to French.

1. Given text in English: "The quick brown fox jumps over the lazy dog."...
2. When the LLM translates the text to French...
3. It should produce: "Le rapide renard brun saute par-dessus le chien paresseux."

_(Don't expect mini orca (GPT4All) to translate this correctly... I got "Le renard noir a le corps de la souris." instead... that's because mini orca isn't trained to translate)_

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = PromptTemplate(
  template="Translate the following English sentence into French: '{text}'",
  input_variables=["text"],
)
chain = prompt | llm | StrOutputParser()

print(chain.invoke({"text": "The quick brown fox jumps over the lazy dog."}))



Le renard brun rapide saute par dessus le chien paresseux.


### Task 1: Summarization

**Task**: Generate a prompt that instructs the language model to summarize a long article.

1. Given a lengthy article
2. When you prompt the LLM to summarize it
3. It should produce an accurate summary

Want to go deeper? [Dig into Hugging Face's tutorial on summarization and learn how you can use a "rouge" score to measure the quality of a model's summarial output](https://huggingface.co/learn/nlp-course/en/chapter7/5?fw=pt#summarization)

#### The article

run this to load the article

In [ ]:
# from: https://github.com/MostlyAdequate/mostly-adequate-guide/blob/master/ch04.md
article = """
# Chapter 04: Currying

## Can't Live If Livin' Is without You
My Dad once explained how there are certain things one can live without until one acquires them. A microwave is one such thing. Smart phones, another. The older folks among us will remember a fulfilling life sans internet. For me, currying is on this list.

The concept is simple: You can call a function with fewer arguments than it expects. It returns a function that takes the remaining arguments.

You can choose to call it all at once or simply feed in each argument piecemeal.

```js
const add = x => y => x + y;
const increment = add(1);
const addTen = add(10);

increment(2); // 3
addTen(2); // 12
```

Here we've made a function `add` that takes one argument and returns a function. By calling it, the returned function remembers the first argument from then on via the closure. Calling it with both arguments all at once is a bit of a pain, however, so we can use a special helper function called `curry` to make defining and calling functions like this easier.

Let's set up a few curried functions for our enjoyment. From now on, we'll summon our `curry`
function defined in the [Appendix A - Essential Function Support](./appendix_a.md).

```js
const match = curry((what, s) => s.match(what));
const replace = curry((what, replacement, s) => s.replace(what, replacement));
const filter = curry((f, xs) => xs.filter(f));
const map = curry((f, xs) => xs.map(f));
```

The pattern I've followed is a simple, but important one. I've strategically positioned the data we're operating on (String, Array) as the last argument. It will become clear as to why upon use.

(The syntax `/r/g`  is a regular expression that means _match every letter 'r'_. Read [more about regular expressions](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Guide/Regular_Expressions) if you like.)

```js
match(/r/g, 'hello world'); // [ 'r' ]

const hasLetterR = match(/r/g); // x => x.match(/r/g)
hasLetterR('hello world'); // [ 'r' ]
hasLetterR('just j and s and t etc'); // null

filter(hasLetterR, ['rock and roll', 'smooth jazz']); // ['rock and roll']

const removeStringsWithoutRs = filter(hasLetterR); // xs => xs.filter(x => x.match(/r/g))
removeStringsWithoutRs(['rock and roll', 'smooth jazz', 'drum circle']); // ['rock and roll', 'drum circle']

const noVowels = replace(/[aeiou]/ig); // (r,x) => x.replace(/[aeiou]/ig, r)
const censored = noVowels('*'); // x => x.replace(/[aeiou]/ig, '*')
censored('Chocolate Rain'); // 'Ch*c*l*t* R**n'
```

What's demonstrated here is the ability to "pre-load" a function with an argument or two in order to receive a new function that remembers those arguments.

I encourage you to clone the Mostly Adequate repository (`git clone
https://github.com/MostlyAdequate/mostly-adequate-guide.git`), copy the code above and have a
go at it in the REPL. The curry function, as well as actually anything defined in the appendixes,
are available in the `support/index.js` module.

Alternatively, have a look at a published version on `npm`:

```
npm install @mostly-adequate/support
```

## More Than a Pun / Special Sauce

Currying is useful for many things. We can make new functions just by giving our base functions some arguments as seen in `hasLetterR`, `removeStringsWithoutRs`, and `censored`.

We also have the ability to transform any function that works on single elements into a function that works on arrays simply by wrapping it with `map`:

```js
const getChildren = x => x.childNodes;
const allTheChildren = map(getChildren);
```

Giving a function fewer arguments than it expects is typically called *partial application*. Partially applying a function can remove a lot of boiler plate code. Consider what the above `allTheChildren` function would be with the uncurried `map` from lodash (note the arguments are in a different order):

```js
const allTheChildren = elements => map(elements, getChildren);
```

We typically don't define functions that work on arrays, because we can just call `map(getChildren)` inline. Same with `sort`, `filter`, and other higher order functions (a *higher order function* is a function that takes or returns a function).

When we spoke about *pure functions*, we said they take 1 input to 1 output. Currying does exactly this: each single argument returns a new function expecting the remaining arguments. That, old sport, is 1 input to 1 output.

No matter if the output is another function - it qualifies as pure. We do allow more than one argument at a time, but this is seen as merely removing the extra `()`'s for convenience.


## In Summary

Currying is handy and I very much enjoy working with curried functions on a daily basis. It is a tool for the belt that makes functional programming less verbose and tedious.

We can make new, useful functions on the fly simply by passing in a few arguments and as a bonus, we've retained the mathematical function definition despite multiple arguments.
"""


#### Your exercise

Perform you work here:

In [ ]:
# TODO

### Task 2: Specifying the Format

**Task**: Building on your solution to Task 1, improve the generated content by specifying the desired format.

1. When you adjust your prompt to recommend that the summary be 3 sentences or less
2. It should adhere to the limit and produce a shorter summary
3. When you adjust your prompt to request bullets instead of prose
4. It should produce a bullet list for the summary

In [ ]:
# TODO

### Task 3: Creative Writing

**Task**: Create a prompt that encourages the model to generate a creative and engaging story opening or poem.

1. When you provide a prompt with a scenario that you want it to write about (e.g. an event in someone's life, an interaction between people, an atmosphere in a time and place, etc.)
2. It should produce a story or poem on the topic you requested

In [ ]:
# TODO

### Task 4: Creating a Persona

**Task**: Building on Task 3, give the LLM cues that influence the writing style by creating a persona.

1. When you
    - give the bot a name (e.g. You are Sean)
    - tell it about it's style and who it is influenced by (choose author(s) whose material was likely included in the training materials for the LLM you are using)
    - then ask it to write about what you asked for before.
2. It should produce material that is believably associated with the persona you provided.
3. (Optional) Include an example in the prompt. Does it improve the output or does it result in _leading the answer_ (when the example is followed too explicitly)? Examples can reduce the quality in some situations, such as creative writing, and improve the quality in other situations, such as code generation, where we desire strong pattern alignment.


In [ ]:
# TODO

### Task 5: Limiting Scope

**Task**: With broad and ambiguous prompts, we sometimes benefit from the LLMs creativity, but it can be difficult to dial in what we're looking for. Building on Tasks 3 or 4, narrow the scope of what you asked the LLM to generate.

1. When you chainge your prompt, providing greater detail and/or constraints (e.g. if you asked the model to generate content about an event, ask it to generate content about a single person in that event, or a moment in time, rather than a long period of time or an entire event.)
2. It should generate content that is more focused.

In [ ]:
# TODO

### Task 6: Code Generation

**Task**: Create a prompt that instructs the model to write tests for for existing code.

1. Given the following function

```py
def now_ms(get_time=time.time):
    return round(get_time() * 1000)
```

2. When you ask the model to generate tests for the given code.
3. It should produce code that can be used to evaluate the quality of the function

_Note: does specifying the test library change the output(e.g. "using pytest")? What happens when you ask for both positive and negative path tests?_#@



In [ ]:
# TODO

## Exercise 2: Retrieval Augmented Generation (RAG)

In this exercise, you will create prompts that combine the retrieval of specific information in order to respond to a prompt using large language models.

Examples are provided for a web loader, but there are [many more to choose from](https://python.langchain.com/docs/modules/data_connection/document_loaders/), as well as different [retrievers](https://python.langchain.com/docs/modules/data_connection/retrievers/), [embedders](https://python.langchain.com/docs/integrations/text_embedding), and [vector stores](https://python.langchain.com/docs/modules/data_connection/vectorstores/).

### Example 1: Q&A

**Task**: Create a prompt that instructs the model to generate a concise answer to a specific question by pulling information from various markdown documents from a GitHub repository.

_This example uses a [WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base) to load the content and passes it to the `load_index` function in this notebook, which will load the content, split it into chunks and then index it in a vector store so it can be used in prompt chains._




In [ ]:
from operator import itemgetter

from langchain.document_loaders import WebBaseLoader
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

def load_95729_md():
    """
    Loads the Markdown documents from the heinz-95729 repository
    """
    def loader(): return WebBaseLoader([
        "https://raw.githubusercontent.com/losandes/heinz-95729/main/Discussion-Board-Policy.md",
        "https://raw.githubusercontent.com/losandes/heinz-95729/main/Project.md",
        "https://raw.githubusercontent.com/losandes/heinz-95729/main/README.md",
    ])
    return load_index(
        "./.indexes/heinz-95729-web",
        loader,
    )

index = load_95729_md()
prompt = ChatPromptTemplate.from_template("""
  Answer the question based only on the following context: {context}
  Answer in the following language: {language}

  Question: {question}""")
chain = (
    {
        "context": itemgetter("question") | index.as_retriever(),
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)
question = "What are the objectives for the course project?"
lang = "English"
print(chain.invoke({"question": question, "language": lang}))

TRACE::20246.793:: using cached index ./.indexes/heinz-95729-web
TRACE::20246.795:: preparing embeddings
TRACE::20247.38:: loading index


Answer: The objectives for the course project are to gather requirements from the client, present the requirements to the client in user story format, plan the approach, agree on a branching strategy, work together as a team to implement the design, use pull-requests and code reviews to merge new code, present the solution to the class, and submit peer evaluations.


### Example 2: Formatted Q&A

**Task**: Create a prompt that instructs the model to generate a formatted answer to a specific question by pulling information from various markdown documents from a GitHub repository.

1. Building on Example 1, request that the answer follows a specific format.

In [ ]:
prompt = ChatPromptTemplate.from_template("""
  You will be provided with context and your job is to
  answer a question using that context. Your answer
  should be no more than 3 sentences long. Your answer
  should use bullets. Answer in the following language: {language}

  Context: {context}

  Question: {question}""")
chain = (
    {
        "context": itemgetter("question") | index.as_retriever(),
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | llm
    | StrOutputParser()
)
question = "What are the objectives for the course project?"
lang = "French"
print(chain.invoke({"question": question, "language": lang}))



Réponse: 
- Réunir des exigences du client (professeur). 
- Présenter les exigences au client sous forme d'histoires utilisateur. 
- Planifier votre approche: conception par Planning Poker.
- Convenir d'une stratégie de ramification. 
- Travailler ensemble en tant qu'équipe pour mettre en œuvre la conception. 
- Utiliser des demandes de tirage et des examens de code pour fusionner de nouveaux codes. 
- Présenter votre solution à la classe. 
- Soumettre des évaluations entre pairs.


### Task 1: Knowledge Synthesis

**Task**: Create a prompt that instructs the model to generate a concise summary of a specific topic by pulling information from various sources or documents.

1. Given: find several different sources (web pages) that discuss the same or similar topics
2. When you use the WebBaseLoader to load content from each of those sources
3. And create a prompt that instructs the model to generate a concise summary of a specific topic shared by those sources
4. It should produce a summary that is informed by the content you provided

In [ ]:
# TODO